# pandas批量拆分与合并Excel
实例:
1. 将一个大excel等份拆分成多个excel
2. 将多个小excel合并成一个大excel并标记来源

In [31]:
work_dir = './data/合并目录/'
split_dir = './data/合并目录/splits'

In [32]:
import os

if not os.path.exists(split_dir):
    os.mkdir(split_dir)

## 0. 读取exce

In [33]:
import pandas as pd

In [34]:
df_source = pd.read_excel('./data/movies/users.xlsx')

In [35]:
df_source.head()

,userId,age,gender,occupation,zip-code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [36]:
df_source.index

RangeIndex(start=0, stop=943, step=1)

In [37]:
df_source.shape  # 查看有几行几列

(943, 5)

# 一. 将一个大excel等份拆分成多个excel
1. 使用df.iloc方法, 将一个大DataFrame拆分成多个小DataFrame
2. 将使用dataFrame.to_excel()保存小excel


**1.计算拆分后的每个excel的行数**

In [38]:
# 将某一列做统计并且将统计结果转换为列表方式一
list(df_source['occupation'].value_counts().keys()[0:])

['student',
 'other',
 'educator',
 'administrator',
 'engineer',
 'programmer',
 'librarian',
 'writer',
 'executive',
 'scientist',
 'artist',
 'technician',
 'marketing',
 'entertainment',
 'healthcare',
 'retired',
 'lawyer',
 'salesman',
 'none',
 'homemaker',
 'doctor']

In [39]:
# 将某一列做统计并且将统计结果转换为列表方式二
list(df_source['occupation'].value_counts().index[0:])

['student',
 'other',
 'educator',
 'administrator',
 'engineer',
 'programmer',
 'librarian',
 'writer',
 'executive',
 'scientist',
 'artist',
 'technician',
 'marketing',
 'entertainment',
 'healthcare',
 'retired',
 'lawyer',
 'salesman',
 'none',
 'homemaker',
 'doctor']

In [40]:
# 这个大excel, 会拆封给这些人
user_names = ['xiao_shuai', 'xiao_wang' ,'xiao_ming' , 'xiao_lei', 'xiao_bo', 'xiao_hong']
# 每个人的任务数目
total_row_count = df_source.shape[0]
split_size = total_row_count / len(user_names)
if total_row_count % len(user_names) !=0:
    split_size += 1
int(split_size)

158

**2.拆分成多个DataFrame**

In [ ]:
df_subs = []